In [ ]:
### load ghcnd-stations

In [1]:
import os

def parse_ghcnd_station_line(line):
    """Parse a line from ghcnd-stations.txt according to fixed-width format.
    
    Format:
    ID            1-11   Character
    LATITUDE     13-20   Real
    LONGITUDE    22-30   Real
    ELEVATION    32-37   Real
    STATE        39-40   Character
    NAME         42-71   Character
    GSN FLAG     73-75   Character
    HCN/CRN FLAG 77-79   Character
    WMO ID       81-85   Character
    """
    return {
        'ID': line[0:11].strip(),
        'LATITUDE': float(line[12:20].strip()),
        'LONGITUDE': float(line[21:30].strip()),
        'ELEVATION': float(line[31:37].strip()),
        'STATE': line[38:40].strip(),
        'NAME': line[41:71].strip(),
        'GSN_FLAG': line[72:75].strip(),
        'HCN_CRN_FLAG': line[76:79].strip(),
        'WMO_ID': line[80:85].strip() if len(line) > 80 else ''
    }

#### read ghcnd-stations.txt
relative_data_path = '../../data/noaa-gov-weather/ghcnd-stations.txt'
current_dir = os.getcwd()
# join the current directory and data path
data_path = os.path.join(current_dir, relative_data_path)

# read the file and load into a list

In [2]:
# Test the parser
stations = []
with open(data_path, 'r') as file:
    lines = file.readlines()
    for line in lines:
        station = parse_ghcnd_station_line(line)
        stations.append(station)

print(stations[:3])

[{'ID': 'ACW00011604', 'LATITUDE': 17.1167, 'LONGITUDE': -61.7833, 'ELEVATION': 10.1, 'STATE': '', 'NAME': 'ST JOHNS COOLIDGE FLD', 'GSN_FLAG': '', 'HCN_CRN_FLAG': '', 'WMO_ID': ''}, {'ID': 'ACW00011647', 'LATITUDE': 17.1333, 'LONGITUDE': -61.7833, 'ELEVATION': 19.2, 'STATE': '', 'NAME': 'ST JOHNS', 'GSN_FLAG': '', 'HCN_CRN_FLAG': '', 'WMO_ID': ''}, {'ID': 'AE000041196', 'LATITUDE': 25.333, 'LONGITUDE': 55.517, 'ELEVATION': 34.0, 'STATE': '', 'NAME': 'SHARJAH INTER. AIRP', 'GSN_FLAG': 'GSN', 'HCN_CRN_FLAG': '', 'WMO_ID': '41196'}]


## Converting stations list to pandas DataFrame

In [4]:
import pandas as pd

col_names = ["ID", "LATITUDE", "LONGITUDE", "ELEVATION", "STATE", "NAME"]
df_stations = pd.DataFrame(stations, columns=col_names)
stations_df = df_stations
stations = None
stations_df

,ID,LATITUDE,LONGITUDE,ELEVATION,STATE,NAME
0,ACW00011604,17.1167,-61.7833,10.1,,ST JOHNS COOLIDGE FLD
1,ACW00011647,17.1333,-61.7833,19.2,,ST JOHNS
2,AE000041196,25.3330,55.5170,34.0,,SHARJAH INTER. AIRP
3,AEM00041194,25.2550,55.3640,10.4,,DUBAI INTL
4,AEM00041217,24.4330,54.6510,26.8,,ABU DHABI INTL
...,...,...,...,...,...,...
118487,ZI000067969,-21.0500,29.3670,861.0,,WEST NICHOLSON
118488,ZI000067975,-20.0670,30.8670,1095.0,,MASVINGO
118489,ZI000067977,-21.0170,31.5830,430.0,,BUFFALO RANGE
118490,ZI000067983,-20.2000,32.6160,1132.0,,CHIPINGE


In [5]:
import geopandas
import shapely
import pyogrio

In [6]:
import geopandas as gpd
import fiona

# Specify the .gdb FOLDER path.
# You might need to list layers first if there is multiple inside.
gdb_relative_path = "../../data/PJM/PJM_Market/p20/fcitc_cross_region.gdb"
gdb_path = os.path.join(current_dir, gdb_relative_path)
# Optional: List layers to find the right name if you aren't sure
print(fiona.listlayers(gdb_path))
layer = fiona.listlayers(gdb_path)[0]
# Read the specific layer (replace 'actual_layer_name' with what you found)
pjm_zones_gdf = gpd.read_file(gdb_path, layer=layer)

['PJM_Market']


/home/alitonia/.config/JetBrains/DataSpell2025.2/projects/workspace/.venv/lib/python3.12/site-packages/pyogrio/raw.py:198: RuntimeWarning: organizePolygons() received a polygon with more than 100 parts. The processing may be really slow.  You can skip the processing by setting METHOD=SKIP, or only make it analyze counter-clock wise parts by setting METHOD=ONLY_CCW if you can assume that the outline of holes is counter-clock wise defined
  return ogr_read(


In [7]:
stations_gdf = geopandas.GeoDataFrame(
    stations_df,
    geometry=geopandas.points_from_xy(stations_df.LONGITUDE, stations_df.LATITUDE),
    crs="EPSG:4326" # Standard lat/lon
)
stations_df = None
layer = None
stations_gdf

,ID,LATITUDE,LONGITUDE,ELEVATION,STATE,NAME,geometry
0,ACW00011604,17.1167,-61.7833,10.1,,ST JOHNS COOLIDGE FLD,POINT (-61.7833 17.1167)
1,ACW00011647,17.1333,-61.7833,19.2,,ST JOHNS,POINT (-61.7833 17.1333)
2,AE000041196,25.3330,55.5170,34.0,,SHARJAH INTER. AIRP,POINT (55.517 25.333)
3,AEM00041194,25.2550,55.3640,10.4,,DUBAI INTL,POINT (55.364 25.255)
4,AEM00041217,24.4330,54.6510,26.8,,ABU DHABI INTL,POINT (54.651 24.433)
...,...,...,...,...,...,...,...
118487,ZI000067969,-21.0500,29.3670,861.0,,WEST NICHOLSON,POINT (29.367 -21.05)
118488,ZI000067975,-20.0670,30.8670,1095.0,,MASVINGO,POINT (30.867 -20.067)
118489,ZI000067977,-21.0170,31.5830,430.0,,BUFFALO RANGE,POINT (31.583 -21.017)
118490,ZI000067983,-20.2000,32.6160,1132.0,,CHIPINGE,POINT (32.616 -20.2)


In [18]:
# Reproject pjm_zones_gdf to match stations_gdf CRS
stations_gdf_projected = stations_gdf.to_crs(pjm_zones_gdf.crs)

stations_in_pjm = geopandas.sjoin(
    stations_gdf_projected,
    pjm_zones_gdf, 
    how="inner", 
    predicate="within"
)
stations_in_pjm

,ID_left,LATITUDE,LONGITUDE,ELEVATION,STATE_left,NAME_left,geometry,index_right,ID_right,NAME_right,...,SOURCE,SOURCEDATE,VAL_METHOD,VAL_DATE,WEBSITE,PEAK_LOAD,AVG_LOAD,YEAR,Shape_Length,Shape_Area
60999,US1DCDC0001,38.9171,-76.9626,17.1,DC,WASHINGTON 3.0 ENE,POINT (-76.9626 38.9171),0,14725,"PJM INTERCONNECTION, LLC",...,"FERC 714, EIA 860, EIA 861, TIGER/Line Shapefi...",2019-06-20 00:00:00+00:00,OTHER,2020-05-15 00:00:00+00:00,http://www.pjm.com/,150670.0,127683.0,2018,2.355385e+07,5.968605e+11
61000,US1DCDC0002,38.9574,-77.0337,85.0,DC,WASHINGTON 3.7 NNW,POINT (-77.0337 38.9574),0,14725,"PJM INTERCONNECTION, LLC",...,"FERC 714, EIA 860, EIA 861, TIGER/Line Shapefi...",2019-06-20 00:00:00+00:00,OTHER,2020-05-15 00:00:00+00:00,http://www.pjm.com/,150670.0,127683.0,2018,2.355385e+07,5.968605e+11
61001,US1DCDC0003,38.9377,-77.0991,91.1,DC,WASHINGTON 5.0 WNW,POINT (-77.0991 38.9377),0,14725,"PJM INTERCONNECTION, LLC",...,"FERC 714, EIA 860, EIA 861, TIGER/Line Shapefi...",2019-06-20 00:00:00+00:00,OTHER,2020-05-15 00:00:00+00:00,http://www.pjm.com/,150670.0,127683.0,2018,2.355385e+07,5.968605e+11
61002,US1DCDC0006,38.8928,-76.9834,21.9,DC,WASHINGTON 2.0 ESE,POINT (-76.9834 38.8928),0,14725,"PJM INTERCONNECTION, LLC",...,"FERC 714, EIA 860, EIA 861, TIGER/Line Shapefi...",2019-06-20 00:00:00+00:00,OTHER,2020-05-15 00:00:00+00:00,http://www.pjm.com/,150670.0,127683.0,2018,2.355385e+07,5.968605e+11
61003,US1DCDC0007,38.8943,-76.9982,25.0,DC,WASHINGTON 1.2 SE,POINT (-76.9982 38.8943),0,14725,"PJM INTERCONNECTION, LLC",...,"FERC 714, EIA 860, EIA 861, TIGER/Line Shapefi...",2019-06-20 00:00:00+00:00,OTHER,2020-05-15 00:00:00+00:00,http://www.pjm.com/,150670.0,127683.0,2018,2.355385e+07,5.968605e+11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117645,USW00094830,41.5886,-83.8014,203.9,OH,TOLEDO EXPRESS AP,POINT (-83.8014 41.5886),0,14725,"PJM INTERCONNECTION, LLC",...,"FERC 714, EIA 860, EIA 861, TIGER/Line Shapefi...",2019-06-20 00:00:00+00:00,OTHER,2020-05-15 00:00:00+00:00,http://www.pjm.com/,150670.0,127683.0,2018,2.355385e+07,5.968605e+11
117648,USW00094846,41.9950,-87.9336,200.6,IL,CHICAGO OHARE INTL AP,POINT (-87.9336 41.995),0,14725,"PJM INTERCONNECTION, LLC",...,"FERC 714, EIA 860, EIA 861, TIGER/Line Shapefi...",2019-06-20 00:00:00+00:00,OTHER,2020-05-15 00:00:00+00:00,http://www.pjm.com/,150670.0,127683.0,2018,2.355385e+07,5.968605e+11
117655,USW00094871,42.1244,-86.4267,196.0,MI,BENTON HARBOR AP,POINT (-86.4267 42.1244),0,14725,"PJM INTERCONNECTION, LLC",...,"FERC 714, EIA 860, EIA 861, TIGER/Line Shapefi...",2019-06-20 00:00:00+00:00,OTHER,2020-05-15 00:00:00+00:00,http://www.pjm.com/,150670.0,127683.0,2018,2.355385e+07,5.968605e+11
117657,USW00094892,41.9144,-88.2464,229.8,IL,WEST CHICAGO DUPAGE AP,POINT (-88.2464 41.9144),0,14725,"PJM INTERCONNECTION, LLC",...,"FERC 714, EIA 860, EIA 861, TIGER/Line Shapefi...",2019-06-20 00:00:00+00:00,OTHER,2020-05-15 00:00:00+00:00,http://www.pjm.com/,150670.0,127683.0,2018,2.355385e+07,5.968605e+11


In [19]:
# Filter the spatially joined dataframe for IDs starting with 'USW'
high_quality_stations = stations_in_pjm[stations_in_pjm['ID_left'].str.startswith('USW')].copy()

print(f"Found {len(high_quality_stations)} high-quality USW stations inside PJM.")
high_quality_stations

Found 199 high-quality USW stations inside PJM.


,ID_left,LATITUDE,LONGITUDE,ELEVATION,STATE_left,NAME_left,geometry,index_right,ID_right,NAME_right,...,SOURCE,SOURCEDATE,VAL_METHOD,VAL_DATE,WEBSITE,PEAK_LOAD,AVG_LOAD,YEAR,Shape_Length,Shape_Area
116036,USW00003733,39.0131,-79.4742,1033.3,WV,ELKINS 21 ENE,POINT (-79.4742 39.0131),0,14725,"PJM INTERCONNECTION, LLC",...,"FERC 714, EIA 860, EIA 861, TIGER/Line Shapefi...",2019-06-20 00:00:00+00:00,OTHER,2020-05-15 00:00:00+00:00,http://www.pjm.com/,150670.0,127683.0,2018,2.355385e+07,5.968605e+11
116037,USW00003739,37.2908,-75.9269,8.8,VA,CAPE CHARLES 5 ENE,POINT (-75.9269 37.2908),0,14725,"PJM INTERCONNECTION, LLC",...,"FERC 714, EIA 860, EIA 861, TIGER/Line Shapefi...",2019-06-20 00:00:00+00:00,OTHER,2020-05-15 00:00:00+00:00,http://www.pjm.com/,150670.0,127683.0,2018,2.355385e+07,5.968605e+11
116038,USW00003749,39.6450,-77.4681,563.0,MD,CAMP DAVID,POINT (-77.4681 39.645),0,14725,"PJM INTERCONNECTION, LLC",...,"FERC 714, EIA 860, EIA 861, TIGER/Line Shapefi...",2019-06-20 00:00:00+00:00,OTHER,2020-05-15 00:00:00+00:00,http://www.pjm.com/,150670.0,127683.0,2018,2.355385e+07,5.968605e+11
116039,USW00003757,38.1417,-76.4292,6.4,MD,ST INIGOES WEBSTER NOLF,POINT (-76.4292 38.1417),0,14725,"PJM INTERCONNECTION, LLC",...,"FERC 714, EIA 860, EIA 861, TIGER/Line Shapefi...",2019-06-20 00:00:00+00:00,OTHER,2020-05-15 00:00:00+00:00,http://www.pjm.com/,150670.0,127683.0,2018,2.355385e+07,5.968605e+11
116041,USW00003759,37.9975,-78.4656,358.7,VA,CHARLOTTESVILLE 2 SSE,POINT (-78.4656 37.9975),0,14725,"PJM INTERCONNECTION, LLC",...,"FERC 714, EIA 860, EIA 861, TIGER/Line Shapefi...",2019-06-20 00:00:00+00:00,OTHER,2020-05-15 00:00:00+00:00,http://www.pjm.com/,150670.0,127683.0,2018,2.355385e+07,5.968605e+11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117645,USW00094830,41.5886,-83.8014,203.9,OH,TOLEDO EXPRESS AP,POINT (-83.8014 41.5886),0,14725,"PJM INTERCONNECTION, LLC",...,"FERC 714, EIA 860, EIA 861, TIGER/Line Shapefi...",2019-06-20 00:00:00+00:00,OTHER,2020-05-15 00:00:00+00:00,http://www.pjm.com/,150670.0,127683.0,2018,2.355385e+07,5.968605e+11
117648,USW00094846,41.9950,-87.9336,200.6,IL,CHICAGO OHARE INTL AP,POINT (-87.9336 41.995),0,14725,"PJM INTERCONNECTION, LLC",...,"FERC 714, EIA 860, EIA 861, TIGER/Line Shapefi...",2019-06-20 00:00:00+00:00,OTHER,2020-05-15 00:00:00+00:00,http://www.pjm.com/,150670.0,127683.0,2018,2.355385e+07,5.968605e+11
117655,USW00094871,42.1244,-86.4267,196.0,MI,BENTON HARBOR AP,POINT (-86.4267 42.1244),0,14725,"PJM INTERCONNECTION, LLC",...,"FERC 714, EIA 860, EIA 861, TIGER/Line Shapefi...",2019-06-20 00:00:00+00:00,OTHER,2020-05-15 00:00:00+00:00,http://www.pjm.com/,150670.0,127683.0,2018,2.355385e+07,5.968605e+11
117657,USW00094892,41.9144,-88.2464,229.8,IL,WEST CHICAGO DUPAGE AP,POINT (-88.2464 41.9144),0,14725,"PJM INTERCONNECTION, LLC",...,"FERC 714, EIA 860, EIA 861, TIGER/Line Shapefi...",2019-06-20 00:00:00+00:00,OTHER,2020-05-15 00:00:00+00:00,http://www.pjm.com/,150670.0,127683.0,2018,2.355385e+07,5.968605e+11


In [11]:
from noaa_sdk import NOAA

,ID,ELEMENT,FIRSTYEAR,LASTYEAR
